In [ ]:
import os, json
import numpy as np
import pandas as pd

In [ ]:
WORKER_ID = "WorkerId"
MTURK_URL = "https://mturk-uncertainty.s3.amazonaws.com/"
CERTAINTY_COLUMNS = ["Answer.certainty1", "Answer.certainty2", "Answer.certainty3", "Answer.certainty4", "Answer.certainty5", "Answer.certainty6", "Answer.certainty7", "Answer.certainty8"]
VIDEO_URL_COLUMNS = ["Input.video1_url", "Input.video2_url", "Input.video3_url", "Input.video4_url", "Input.video5_url", "Input.video6_url", "Input.video7_url", "Input.video8_url"]

In [ ]:
qualified_annotators = pd.read_csv("qualified.csv")[WORKER_ID]
qualified_annotator_list = qualified_annotators.to_list()
print(qualified_annotators)

In [ ]:
full_data = []
for batch_filename in ["batch2_results.csv", "batch3_results.csv", "batch4_results.csv", "batch5_results.csv"]:
    data = pd.read_csv(batch_filename)
    useful_data = data[data[WORKER_ID].isin(qualified_annotator_list)]
    full_data.append(useful_data)
full_data = pd.concat(full_data)
print(f"We have {len(full_data) * 8} data points, we were expecting 6480{', so everything is fine' if len(full_data) * 8 == 6480 else ' there is something wrong!'}")

In [ ]:
lower_range, upper_range = [], []
annotator_low_range, annotator_upper_range = {}, {}
annotator_ids = {}

for i, annotator in enumerate(qualified_annotator_list):
    annotations = full_data[full_data[WORKER_ID] == annotator]

    full_annotations = [] 
    for column in CERTAINTY_COLUMNS:
        full_annotations.extend(annotations[column].to_list())

    # full_annotations = np.array(full_annotations)
    # full_annotations = (full_annotations - np.min(full_annotations)) / (np.max(full_annotations) - np.min(full_annotations)) 
    annotator_low_range[annotator] = np.min(full_annotations)
    annotator_upper_range[annotator] = np.max(full_annotations)

    lower_range.append(np.min(full_annotations))
    upper_range.append(np.max(full_annotations))
    annotator_ids[annotator] = i 

    # print(f"{annotator}: {np.min(full_annotations)} - {np.max(full_annotations)} ({np.max(full_annotations)})")
    print(f"{annotator}: {len(full_annotations)}")

print(np.mean(lower_range), np.mean(upper_range))
print(np.median(lower_range), np.median(upper_range))

print(annotator_low_range)
print(annotator_upper_range)
print(len(qualified_annotator_list))

In [ ]:
for video_column_name, certainty_column_name in zip(VIDEO_URL_COLUMNS, CERTAINTY_COLUMNS):
    for index, row in full_data[[video_column_name, certainty_column_name, WORKER_ID]].iterrows():
        video_name = row[video_column_name].replace(MTURK_URL, "")
        annoator_name = row[WORKER_ID]
        with open(os.path.join("data", "JSON-copy", video_name.replace('mp4', 'json')), "r") as file:
            data = json.load(file)
            data['annotator'] = row[WORKER_ID]
            data['annotator_id'] = annotator_ids[row[WORKER_ID]]
            data['confidence'] = row[certainty_column_name]
            # data['confidence'] = (row[certainty_column_name] - annotator_low_range[annoator_name]) / (annotator_upper_range[annoator_name] - annotator_low_range[annoator_name])
    
        with open(os.path.join("data", "JSON", video_name.replace('mp4', 'json')), "w") as file:
            json.dump(data, file)